In [86]:
import pandas as pd
from scipy.integrate import simps
import numpy as np

In [87]:
# reading NASA DATA

cpu_nasa = np.load("/home/matteo/Documents/postDoc/CTTC/FGCS/git_repo/NASA_turbofan_regression/results/deployability/processed/cpu_usage_data.npz")
gpu_nasa = np.load("/home/matteo/Documents/postDoc/CTTC/FGCS/git_repo/NASA_turbofan_regression/results/deployability/processed/gpu_usage_data.npz")
memory_usage_nasa = np.load("/home/matteo/Documents/postDoc/CTTC/FGCS/git_repo/NASA_turbofan_regression/results/deployability/processed/memory_usage_data.npz")
times_pred_nasa = np.load("/home/matteo/Documents/postDoc/CTTC/FGCS/git_repo/NASA_turbofan_regression/results/deployability/processed/times_pred_data.npz")
times_train_nasa = np.load("/home/matteo/Documents/postDoc/CTTC/FGCS/git_repo/NASA_turbofan_regression/results/deployability/processed/times_train_data.npz")

In [88]:
# avg_train_time_lstm_gpu=avg_train_time_lstm_gpu,
# avg_train_time_lstm_cpu=avg_train_time_lstm_cpu,
# avg_train_time_ecae_1layers_mlp=avg_train_time_ecae_1layers_mlp,
# avg_train_time_ecae_2layers_mlp=avg_train_time_ecae_2layers_mlp,
# avg_train_time_ecae_1layers_ae=avg_train_time_ecae_1layers_ae,
# avg_train_time_ecae_2layers_ae=avg_train_time_ecae_2layers_ae,
# avg_train_time_race_step1_1layers=avg_train_time_race_step1_1layers,
# avg_train_time_race_step1_2layers=avg_train_time_race_step1_2layers,
# avg_train_time_race_step2_1layers=avg_train_time_race_step2_1layers,
# avg_train_time_race_step2_2layers=avg_train_time_race_step2_2layers,

In [89]:
avg_cpu_lstm_gpu = cpu_nasa["avg_cpu_usage_lstm_gpu"]
avg_cpu_lstm_cpu = cpu_nasa["avg_cpu_usage_lstm_cpu"]
avg_cpu_ecae_1layers_mlp = cpu_nasa["avg_cpu_usage_ecae_1layers_mlp"]
avg_cpu_ecae_2layers_mlp = cpu_nasa["avg_cpu_usage_ecae_2layers_mlp"]
avg_cpu_ecae_1layers_ae = cpu_nasa["avg_cpu_usage_ecae_1layers_ae"]
avg_cpu_ecae_2layers_ae = cpu_nasa["avg_cpu_usage_ecae_2layers_ae"]
avg_cpu_race_step2_1layers=cpu_nasa["avg_cpu_usage_race_step2_1layers"]
avg_cpu_race_step2_2layers=cpu_nasa["avg_cpu_usage_race_step2_2layers"]

In [90]:
def getResourceUsage(df_energy, startTime, endTime):
    # select portion from df_energy with timestamp between start_train and end_train
    df_energy_filtered = df_energy[(df_energy["timestamp"] >= startTime) & (df_energy["timestamp"] <= endTime)]
    print(len(df_energy_filtered), "rows in df_energy_filtered")
    print("monitoring stated ealier than startTime:", df_energy["timestamp"].min() < startTime)
    print("monitoring ended later than endTime:", df_energy["timestamp"].max() > endTime)
    print("duration", endTime - startTime)

    # calculate the mean and std of each column
    cpu_usage_percentage_mean = df_energy_filtered["energon_cpu_total_usage_percentage"].mean()
    cpu_usage_percentage_std = df_energy_filtered["energon_cpu_total_usage_percentage"].std()
    gpu_usage_percentage_mean = df_energy_filtered["energon_gpu_total_usage_percentage"].mean()
    gpu_usage_percentage_std = df_energy_filtered["energon_gpu_total_usage_percentage"].std()
    memory_usage_mean = df_energy_filtered["energon_ram_used_percentage"].mean()
    memory_usage_std = df_energy_filtered["energon_ram_used_percentage"].std()
    total_power = df_energy_filtered["energon_total_in_power_mW"].mean()
    max_cpu_power = df_energy_filtered["energon_cpu_in_power_mW"].max()
    max_gpu_power = df_energy_filtered["energon_gpu_in_power_mW"].max()

    # calculate the energy consumption using simpson's rule
    total_energy_consumption = simps(df_energy_filtered["energon_total_in_power_mW"], df_energy_filtered["timestamp"]) / 1000  # in Joules
    cpu_energy_consumption = simps(df_energy_filtered["energon_cpu_in_power_mW"], df_energy_filtered["timestamp"]) / 1000
    gpu_energy_consumption = simps(df_energy_filtered["energon_gpu_in_power_mW"], df_energy_filtered["timestamp"]) / 1000

    return {
        "tot_time": endTime - startTime,
        "cpu_usage_percentage_mean": cpu_usage_percentage_mean,
        "cpu_usage_percentage_std": cpu_usage_percentage_std,
        "gpu_usage_percentage_mean": gpu_usage_percentage_mean,
        "gpu_usage_percentage_std": gpu_usage_percentage_std,
        "memory_usage_mean": memory_usage_mean,
        "memory_usage_std": memory_usage_std,
        "max_cpu_power": max_cpu_power,
        "max_gpu_power": max_gpu_power,
        "total_energy_consumption": total_energy_consumption,
        "cpu_energy_consumption": cpu_energy_consumption,
        "gpu_energy_consumption": gpu_energy_consumption
    }



In [91]:
df_energy_ecae = pd.read_csv("./ECAE.csv", index_col=False)
df_energy_race = pd.read_csv("./RACE_step2.csv", index_col=False)
df_energy_ae_mfcc = pd.read_csv("./AE_MFCC13.csv", index_col=False)
df_energy_ae_mel = pd.read_csv("./AE_MELSPECTRO.csv", index_col=False)

# count the number of rows with -1 values
print("Number of rows with -1 values in df_energy_ecae:", len(df_energy_ecae[df_energy_ecae.eq(-1).any(axis=1)]))
print("Number of rows with -1 values in df_energy race:", len(df_energy_race[df_energy_race.eq(-1).any(axis=1)]))
print("Number of rows with -1 values in df_energy_ae_mfcc:", len(df_energy_ae_mfcc[df_energy_ae_mfcc.eq(-1).any(axis=1)]))
print("Number of rows with -1 values in df_energy_ae_mel:", len(df_energy_ae_mel[df_energy_ae_mel.eq(-1).any(axis=1)]))

# remove lines containing -1 values in any column
df_energy_ecae = df_energy_ecae[df_energy_ecae.ge(0).all(axis=1)]
df_energy_race = df_energy_race[df_energy_race.ge(0).all(axis=1)]
df_energy_ae_mfcc = df_energy_ae_mfcc[df_energy_ae_mfcc.ge(0).all(axis=1)]
df_energy_ae_mel = df_energy_ae_mel[df_energy_ae_mel.ge(0).all(axis=1)]


Number of rows with -1 values in df_energy_ecae: 4
Number of rows with -1 values in df_energy race: 28
Number of rows with -1 values in df_energy_ae_mfcc: 6
Number of rows with -1 values in df_energy_ae_mel: 8


In [92]:
df_energy_ecae.columns

Index(['energon_total_in_power_mW', 'energon_cpu_in_power_mW',
       'energon_gpu_in_power_mW', 'energon_cpu_total_usage_percentage',
       'energon_gpu_total_usage_percentage', 'energon_ram_used_percentage',
       'timestamp'],
      dtype='object')

In [93]:
DEV_ASSETS = ['fan', 'gearbox', 'slider', 'ToyCar',  'bearing', 'valve', 'ToyTrain']
EVAL_ASSETS = ["bandsaw", "grinder", "shaker", "ToyDrone", "ToyNscale", "ToyTank", "Vacuum"]

ALL_ASSETS = DEV_ASSETS + EVAL_ASSETS 

## Prediction time

In [94]:
dfs_ae_melspec = []
dfs_ae_mfcc = []
dfs_ecae_layer1 = []
dfs_ecae_layer2 = []
dfs_race_layer1 = []
dfs_race_layer2 = []


In [95]:
for a in ALL_ASSETS:
    # ae melspec
    df_ae_melspec = pd.read_csv(f"../accuracy/ae_baseline_melspec/{a}_results.csv", index_col=False)
    df_ae_mfcc = pd.read_csv(f"../accuracy/ae_baseline_mfcc13/{a}_results.csv", index_col=False)
    df_ecae_layer1 = pd.read_csv(f"../deployability/ecae_loading_hps/1_rc_layers/cae_pytorch_rc_mlp_anomaly_{a}/{a}_results_train_cae_mlp_load_rc_hps.csv", index_col=False)
    df_ecae_layer2 = pd.read_csv(f"../deployability/ecae_loading_hps/2_rc_layers/cae_pytorch_rc_mlp_anomaly_{a}/{a}_results_train_cae_mlp_load_rc_hps.csv", index_col=False)
    df_race_layer1 = pd.read_csv(f"../accuracy/direct_test_step2/1/{a}_results.csv", index_col=False)
    df_race_layer2 = pd.read_csv(f"../accuracy/direct_test_step2/2/{a}_results.csv", index_col=False)

    dfs_ae_melspec.append(df_ae_melspec)
    dfs_ae_mfcc.append(df_ae_mfcc)
    dfs_ecae_layer1.append(df_ecae_layer1)
    dfs_ecae_layer2.append(df_ecae_layer2)
    dfs_race_layer1.append(df_race_layer1)
    dfs_race_layer2.append(df_race_layer2)

In [ ]:
dcase_melspec_train_time = []
dcase_melspec_pred_time = []
dcase_melspec_cpu_usage = []
dcase_melspec_gpu_usage = []
dcase_melspec_memory_usage = []
dcase_melspec_energy = []

dcase_mfcc_train_time = []
dcase_mfcc_pred_time = []
dcase_mfcc_cpu_usage = []
dcase_mfcc_gpu_usage = []
dcase_mfcc_memory_usage = []
dcase_mfcc_energy = []

dcase_ecae_layer1_train_time_cnn = []
dcase_ecae_layer1_pred_time_cnn = []
dcase_ecae_layer1_train_time_mlp = []
dcase_ecae_layer1_pred_time_mlp = []
dcase_ecae_layer1_cpu_usage_cnn = []
dcase_ecae_layer1_cpu_usage_mlp = []
dcase_ecae_layer1_gpu_usage_cnn = []
dcase_ecae_layer1_gpu_usage_mlp = []
dcase_ecae_layer1_memory_usage_cnn = []
dcase_ecae_layer1_memory_usage_mlp = []
dcase_ecae_layer1_energy_cnn = []
dcase_ecae_layer1_energy_mlp = []

dcase_ecae_layer2_train_time_cnn = []
dcase_ecae_layer2_pred_time_cnn = []
dcase_ecae_layer2_train_time_mlp = []
dcase_ecae_layer2_pred_time_mlp = []
dcase_ecae_layer2_cpu_usage_cnn = []
dcase_ecae_layer2_cpu_usage_mlp = []
dcase_ecae_layer2_gpu_usage_cnn = []
dcase_ecae_layer2_gpu_usage_mlp = []
dcase_ecae_layer2_memory_usage_cnn = []
dcase_ecae_layer2_memory_usage_mlp = []
dcase_ecae_layer2_energy_cnn = []
dcase_ecae_layer2_energy_mlp = []

dcase_race_layer1_train_time = []
dcase_race_layer1_pred_time = []
dcase_race_layer1_cpu_usage = []
dcase_race_layer1_gpu_usage = []
dcase_race_layer1_memory_usage = []
dcase_race_layer1_energy = []

dcase_race_layer2_train_time = []
dcase_race_layer2_pred_time = []
dcase_race_layer2_cpu_usage = []
dcase_race_layer2_gpu_usage = []
dcase_race_layer2_memory_usage = []
dcase_race_layer2_energy = []

In [97]:
dfs_ecae_layer1[0].columns

Index(['max_timesteps_for_cae_padding', 'original_feature_dim_N_MFCC',
       'cae_training_time_seconds', 'start_cae_training_time',
       'end_cae_training_time', 'cae_latent_dim', 'macs_cae', 'n_params_cae',
       'cae_encoding_time_seconds', 'start_cae_encoding_time',
       'end_cae_encoding_time', 'RC_L1_units', 'RC_L1_RC_node_type',
       'RC_L1_sr', 'RC_L1_lr', 'RC_L1_input_scaling', 'RC_L1_connectivity',
       'RC_L1_activation', 'final_mlp_training_time_seconds',
       'start_final_mlp_training_time', 'end_final_mlp_training_time',
       'mlp_model_size_bytes', 'macs_mlp_readout', 'n_params_mlp_readout',
       'final_mlp_reconstruction_time_seconds',
       'start_final_mlp_reconstruction_time',
       'end_final_mlp_reconstruction_time',
       'anomaly_threshold_on_latent_error', 'true_positive', 'true_negative',
       'false_positive', 'false_negative', 'auc_score', 'pauc_score',
       'accuracy', 'macro_avg_precision', 'macro_avg_recall', 'macro_avg_f1',
       '

In [98]:
dfs_ae_melspec[0]["end_prediction_time"]

0    1.748171e+09
Name: end_prediction_time, dtype: float64

In [ ]:
for a in ALL_ASSETS:
    print(f"Processing {a}...")
    mels_train = getResourceUsage(df_energy_ae_mel, dfs_ae_melspec[ALL_ASSETS.index(a)]["start_train_time"].iloc[0], dfs_ae_melspec[ALL_ASSETS.index(a)]["end_train_time"].iloc[0])
    mfcc_train = getResourceUsage(df_energy_ae_mfcc, dfs_ae_mfcc[ALL_ASSETS.index(a)]["start_train_time"].iloc[0], dfs_ae_mfcc[ALL_ASSETS.index(a)]["end_train_time"].iloc[0])    
    ecae_layer1_cnn = getResourceUsage(df_energy_ecae, dfs_ecae_layer1[ALL_ASSETS.index(a)]["start_cae_training_time"].iloc[0], dfs_ecae_layer1[ALL_ASSETS.index(a)]["end_cae_training_time"].iloc[0])
    ecae_layer1_mlp = getResourceUsage(df_energy_ecae, dfs_ecae_layer1[ALL_ASSETS.index(a)]["start_final_mlp_training_time"].iloc[0], dfs_ecae_layer1[ALL_ASSETS.index(a)]["end_final_mlp_training_time"].iloc[0])
    ecae_layer2_cnn = getResourceUsage(df_energy_ecae, dfs_ecae_layer2[ALL_ASSETS.index(a)]["start_cae_training_time"].iloc[0], dfs_ecae_layer2[ALL_ASSETS.index(a)]["end_cae_training_time"].iloc[0])
    ecae_layer2_mlp = getResourceUsage(df_energy_ecae, dfs_ecae_layer2[ALL_ASSETS.index(a)]["start_final_mlp_training_time"].iloc[0], dfs_ecae_layer2[ALL_ASSETS.index(a)]["end_final_mlp_training_time"].iloc[0])
    
    race_layer1_train = getResourceUsage(df_energy_race, dfs_race_layer1[ALL_ASSETS.index(a)]["start_training_time"].iloc[0], dfs_race_layer1[ALL_ASSETS.index(a)]["end_training_time"].iloc[0])
    race_layer2_train = getResourceUsage(df_energy_race, dfs_race_layer2[ALL_ASSETS.index(a)]["start_training_time"].iloc[0], dfs_race_layer2[ALL_ASSETS.index(a)]["end_training_time"].iloc[0])
    
    dcase_melspec_train_time.append(dfs_ae_melspec[ALL_ASSETS.index(a)]["train_time_s"].iloc[0])
    dcase_melspec_pred_time.append(dfs_ae_melspec[ALL_ASSETS.index(a)]["pred_time_s"].iloc[0])
    dcase_mfcc_train_time.append(dfs_ae_mfcc[ALL_ASSETS.index(a)]["train_time_seconds"].iloc[0])
    dcase_mfcc_pred_time.append(dfs_ae_mfcc[ALL_ASSETS.index(a)]["predict_time_seconds"].iloc[0])

    dcase_ecae_layer1_train_time_cnn.append(dfs_ecae_layer1[ALL_ASSETS.index(a)]["cae_training_time_seconds"].iloc[0])
    dcase_ecae_layer1_train_time_mlp.append(dfs_ecae_layer1[ALL_ASSETS.index(a)]["final_mlp_training_time_seconds"].iloc[0])
    dcase_ecae_layer1_pred_time_cnn.append(dfs_ecae_layer1[ALL_ASSETS.index(a)]["cae_encoding_time_seconds"].iloc[0])
    dcase_ecae_layer1_pred_time_mlp.append(dfs_ecae_layer1[ALL_ASSETS.index(a)]["final_mlp_reconstruction_time_seconds"].iloc[0])

    dcase_ecae_layer2_train_time_cnn.append(dfs_ecae_layer2[ALL_ASSETS.index(a)]["cae_training_time_seconds"].iloc[0])
    dcase_ecae_layer2_train_time_mlp.append(dfs_ecae_layer2[ALL_ASSETS.index(a)]["final_mlp_training_time_seconds"].iloc[0])
    dcase_ecae_layer2_pred_time_cnn.append(dfs_ecae_layer2[ALL_ASSETS.index(a)]["cae_encoding_time_seconds"].iloc[0])
    dcase_ecae_layer2_pred_time_mlp.append(dfs_ecae_layer2[ALL_ASSETS.index(a)]["final_mlp_reconstruction_time_seconds"].iloc[0])

    dcase_race_layer1_train_time.append(dfs_race_layer1[ALL_ASSETS.index(a)]["mlp_training_time"].iloc[0])
    dcase_race_layer1_pred_time.append(dfs_race_layer1[ALL_ASSETS.index(a)]["mlp_prediction_time"].iloc[0])
    dcase_race_layer2_train_time.append(dfs_race_layer2[ALL_ASSETS.index(a)]["mlp_training_time"].iloc[0])
    dcase_race_layer2_pred_time.append(dfs_race_layer2[ALL_ASSETS.index(a)]["mlp_prediction_time"].iloc[0])


    dcase_melspec_cpu_usage.append(mels_train["cpu_usage_percentage_mean"])
    dcase_melspec_gpu_usage.append(mels_train["gpu_usage_percentage_mean"])
    dcase_melspec_memory_usage.append(mels_train["memory_usage_mean"])
    dcase_melspec_energy.append(mels_train["total_energy_consumption"])

    dcase_mfcc_cpu_usage.append(mfcc_train["cpu_usage_percentage_mean"])
    dcase_mfcc_gpu_usage.append(mfcc_train["gpu_usage_percentage_mean"])
    dcase_mfcc_memory_usage.append(mfcc_train["memory_usage_mean"])
    dcase_mfcc_energy.append(mfcc_train["total_energy_consumption"])

    dcase_ecae_layer1_cpu_usage_cnn.append(ecae_layer1_cnn["cpu_usage_percentage_mean"])
    dcase_ecae_layer1_gpu_usage_cnn.append(ecae_layer1_cnn["gpu_usage_percentage_mean"])
    dcase_ecae_layer1_memory_usage_cnn.append(ecae_layer1_cnn["memory_usage_mean"])
    dcase_ecae_layer1_energy_cnn.append(ecae_layer1_cnn["total_energy_consumption"])
    dcase_ecae_layer1_cpu_usage_mlp.append(ecae_layer1_mlp["cpu_usage_percentage_mean"])
    dcase_ecae_layer1_gpu_usage_mlp.append(ecae_layer1_mlp["gpu_usage_percentage_mean"])
    dcase_ecae_layer1_memory_usage_mlp.append(ecae_layer1_mlp["memory_usage_mean"])
    dcase_ecae_layer1_energy_mlp.append(ecae_layer1_mlp["total_energy_consumption"])

    dcase_ecae_layer2_cpu_usage_cnn.append(ecae_layer2_cnn["cpu_usage_percentage_mean"])
    dcase_ecae_layer2_gpu_usage_cnn.append(ecae_layer2_cnn["gpu_usage_percentage_mean"])
    dcase_ecae_layer2_memory_usage_cnn.append(ecae_layer2_cnn["memory_usage_mean"])
    dcase_ecae_layer2_energy_cnn.append(ecae_layer2_cnn["total_energy_consumption"])
    dcase_ecae_layer2_cpu_usage_mlp.append(ecae_layer2_mlp["cpu_usage_percentage_mean"])
    dcase_ecae_layer2_gpu_usage_mlp.append(ecae_layer2_mlp["gpu_usage_percentage_mean"])
    dcase_ecae_layer2_memory_usage_mlp.append(ecae_layer2_mlp["memory_usage_mean"])
    dcase_ecae_layer2_energy_mlp.append(ecae_layer2_mlp["total_energy_consumption"])

    dcase_race_layer1_cpu_usage.append(race_layer1_train["cpu_usage_percentage_mean"])
    dcase_race_layer1_gpu_usage.append(race_layer1_train["gpu_usage_percentage_mean"])
    dcase_race_layer1_memory_usage.append(race_layer1_train["memory_usage_mean"])
    dcase_race_layer1_energy.append(race_layer1_train["total_energy_consumption"])

    dcase_race_layer2_cpu_usage.append(race_layer2_train["cpu_usage_percentage_mean"])
    dcase_race_layer2_gpu_usage.append(race_layer2_train["gpu_usage_percentage_mean"])
    dcase_race_layer2_memory_usage.append(race_layer2_train["memory_usage_mean"])
    dcase_race_layer2_energy.append(race_layer2_train["total_energy_consumption"])
    

Processing fan...
1022 rows in df_energy_filtered
monitoring stated ealier than startTime: True
monitoring ended later than endTime: True
duration 512.5418264865875
992 rows in df_energy_filtered
monitoring stated ealier than startTime: True
monitoring ended later than endTime: True
duration 498.7475550174713
4 rows in df_energy_filtered
monitoring stated ealier than startTime: True
monitoring ended later than endTime: True
duration 2.145561695098877
38 rows in df_energy_filtered
monitoring stated ealier than startTime: True
monitoring ended later than endTime: True
duration 19.42101740837097
4 rows in df_energy_filtered
monitoring stated ealier than startTime: True
monitoring ended later than endTime: True
duration 2.2070963382720947
80 rows in df_energy_filtered
monitoring stated ealier than startTime: True
monitoring ended later than endTime: True
duration 39.45014834403992
44 rows in df_energy_filtered
monitoring stated ealier than startTime: True
monitoring ended later than endTim